### 0. Unnecessary thing


In [ ]:
!sudo rm -r -f '/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion/test512'
!sudo rm -r -f '/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion/train512'
!sudo rm -r -f '/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion/train512_vbd_train_data'
!sudo rm -r -f '/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion/train512_to_train'

In [ ]:
!unzip '/content/drive/MyDrive/archive.zip' -d '/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2'

In [ ]:
!ls '/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion/test512'

### 1. Import library and read file csv

In [ ]:
import numpy as np
import pandas as pd
import cv2
from ast import literal_eval
import cv2
import matplotlib.pyplot as plt

import glob

train_df = pd.read_csv("/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/train.csv")
train_df.columns =['image_id','x_min', 'y_min',	'x_max', 'y_max','class_id']

train_df.head()

In [ ]:
val_df = pd.read_csv("/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/val.csv")
val_df.columns =['image_id','x_min', 'y_min',	'x_max', 'y_max','class_id']

val_df.head()

In [ ]:
train_df['x_mid'] = train_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train_df['y_mid'] = train_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

train_df['w'] = train_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
train_df['h'] = train_df.apply(lambda row: (row.y_max-row.y_min), axis =1)
train_df.head()

In [ ]:
val_df['x_mid'] = val_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
val_df['y_mid'] = val_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

val_df['w'] = val_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
val_df['h'] = val_df.apply(lambda row: (row.y_max-row.y_min), axis =1)
val_df.head()

In [ ]:
fr = cv2.imread("/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/train512/051132a778e61a86eb147c7c6f564dfe.png")
frame_rgb = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
frame_rgb = cv2.rectangle(frame_rgb, (324, 160), (324 + 69, 160 + 60), (255, 0, 0), 5)
cv2.putText(frame_rgb, 'Aortic enlargement', (324, 160 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 0, 0), 2)
plt.imshow(frame_rgb)

### 2. Convert data

In [ ]:
train_df['x_min'] = train_df.apply(lambda row: row.x_min/512, axis =1)
train_df['y_min'] = train_df.apply(lambda row: row.y_min/512, axis =1)

train_df['x_max'] = train_df.apply(lambda row: row.x_max/512, axis =1)
train_df['y_max'] = train_df.apply(lambda row: row.y_max/512, axis =1)

train_df['x_mid'] = train_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train_df['y_mid'] = train_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

train_df['w'] = train_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
train_df['h'] = train_df.apply(lambda row: (row.y_max-row.y_min), axis =1)

train_df['area'] = train_df['w']*train_df['h']
train_df.head()

In [ ]:
val_df['x_min'] = val_df.apply(lambda row: row.x_min/512, axis =1)
val_df['y_min'] = val_df.apply(lambda row: row.y_min/512, axis =1)

val_df['x_max'] = val_df.apply(lambda row: row.x_max/512, axis =1)
val_df['y_max'] = val_df.apply(lambda row: row.y_max/512, axis =1)

val_df['x_mid'] = val_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
val_df['y_mid'] = val_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

val_df['w'] = val_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
val_df['h'] = val_df.apply(lambda row: (row.y_max-row.y_min), axis =1)

val_df['area'] = val_df['w']*train_df['h']
val_df.head()

### 3. Check BBox

In [ ]:
def convertYoloToPascal(size, coord):
    x_max = int(((2*size[0]*float(coord[0]))+(size[0]*float(coord[2])))/2)
    x_min = int(((2*size[0]*float(coord[0]))-(size[0]*float(coord[2])))/2)

    y_max = int(((2*size[1]*float(coord[1]))+(size[1]*float(coord[3])))/2)
    y_min = int(((2*size[1]*float(coord[1]))-(size[1]*float(coord[3])))/2)
    return (x_min,y_min,x_max,y_max)

convertYoloToPascal((512, 512), [0.633789, 0.312500, 0.134766, 0.117188])

(289, 129, 359, 190)

In [ ]:
def midPoint(x_min, y_min, x_max, y_max):
    x_mid = (x_max + x_min)//2
    y_mid = (y_max + y_min)//2
    w = (x_max - x_min)
    h = (y_max - y_min)
    return (x_mid, y_mid, w, h)

midPoint(289, 129, 359, 190)

(324, 159, 70, 61)

In [ ]:
fr = cv2.imread("/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion/train512/051132a778e61a86eb147c7c6f564dfe.png")
frame_rgb = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
frame_rgb = cv2.rectangle(frame_rgb, (324, 159), (324 + 70, 159 + 61), (255, 0, 0), 5)
cv2.putText(frame_rgb, 'Aortic enlargement', (324, 160 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 0, 0), 2)
plt.imshow(frame_rgb)

### 4. Make file txt yolo label

In [ ]:
classes_df = pd.read_csv("/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/classes.csv", names = ["classes", "Id"])
classes_df.head()
classes_dict = dict()
for i in range(len(classes_df)):
  classes_dict[classes_df["classes"][i]]= classes_df["Id"][i]
print(classes_dict)

In [ ]:
import os.path
from os import path
from pathlib import Path
train_df['yolo_box'] = train_df[['x_min', 'y_min', 'w', 'h']].values.tolist()

print("We have {} unique images with boxes.".format(len(train_df.image_id.unique())))
unique_img_ids = train_df.image_id.unique()

folder_location = "/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/train512_vbd_train_data"

Path(folder_location).mkdir(parents=True, exist_ok=True)
Path(folder_location+"/train512").mkdir(parents=True, exist_ok=True)
Path("/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/train512_to_train").mkdir(parents=True, exist_ok=True)

for img_id in unique_img_ids: # loop through all unique image ids. Remove the slice to do all images
    filt_df = train_df.query("image_id == @img_id") # filter the df to a specific id
    #all_boxes = filt_df.yolo_box.value
    #print(img_id)
    file_img_name = "/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/{}".format(img_id)
    file_name = "{}/{}.txt".format(folder_location,img_id.split('.')[0]) # specify the name of the folder and get a file name
    print(file_name)
    try:
      Path(file_img_name).rename("/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/train512_to_train/{}".format(img_id.split('/')[1]))
    except:
      print("File has been moved!")
    with open(file_name, 'w+') as file: # append lines to file
        for i in filt_df.iterrows():
            class_name = i[1].class_id # get name of class
            class_id = classes_dict[class_name] # get Id of class
            s = f"{class_id} %s %s %s %s \n" # The first number is the class name
            new_line = (s % tuple(i[1].yolo_box))
            file.write(new_line)

In [ ]:
val_df['yolo_box'] = val_df[['x_min', 'y_min', 'w', 'h']].values.tolist()

print("We have {} unique images with boxes.".format(len(val_df.image_id.unique())))
unique_img_ids = val_df.image_id.unique()

folder_location = "/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/val512_vbd_val_data"

Path(folder_location).mkdir(parents=True, exist_ok=True)
Path("/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/val512").mkdir(parents=True, exist_ok=True)

for img_id in unique_img_ids: # loop through all unique image ids. Remove the slice to do all images
    filt_df = train_df.query("image_id == @img_id") # filter the df to a specific id
    #all_boxes = filt_df.yolo_box.value
    #print(img_id)
    file_img_name = "/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/{}".format(img_id)
    file_name = "{}/{}.txt".format(folder_location,img_id.split('.')[0]) # specify the name of the folder and get a file name
    print(file_img_name)
    print(file_name)

    try:
      Path(file_img_name).rename("/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/val512/{}".format(img_id.split('/')[1]))
    except:
      print("File has been moved!")
    with open(file_name, 'w+') as file: # append lines to file
        for i in filt_df.iterrows():
            class_name = i[1].class_id # get name of class
            class_id = classes_dict[class_name] # get Id of class
            s = f"{class_id} %s %s %s %s \n" # The first number is the class name
            new_line = (s % tuple(i[1].yolo_box))
            file.write(new_line)

In [ ]:
!ls '/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/train512'

In [ ]:
import os.path
path = "/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/val512"
num_files = len([f for f in os.listdir(path)if os.path.isfile(os.path.join(path, f))])
print(num_files)

879


In [ ]:
import os.path
path = "/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/train512"
num_files = len([f for f in os.listdir(path)if os.path.isfile(os.path.join(path, f))])
print(num_files)

10606


In [ ]:
import os.path
path = "/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/train512_to_train"
num_files = len([f for f in os.listdir(path)if os.path.isfile(os.path.join(path, f))])
print(num_files)

3515


### 5. Zip the label



In [ ]:
# zip to make files easier to download

!zip -r yolo_labels_train.zip "/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/train512_vbd_train_data/train512"
!zip -r yolo_labels_val.zip "/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/val512_vbd_val_data/train512"

In [ ]:


!zip -r img_train.zip "/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/train512_to_train"
!zip -r img_val.zip "/content/drive/MyDrive/VinBigData Chest X-ray Abnormalities Detetion 2/val512"

In [ ]:
# !mv '/content/img_train.zip' '/content/drive/MyDrive/'
%mv '/content/img_val.zip' '/content/drive/MyDrive/img_val.zip'
%mv '/content/yolo_labels_train.zip' '/content/drive/MyDrive/yolo_labels_train.zip'
%mv '/content/yolo_labels_val.zip' '/content/drive/MyDrive/yolo_labels_val.zip'